<a href="https://colab.research.google.com/github/yuzuponikemi/LLMpg/blob/main/LoRA-slack-persona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ==============================================================================
# LoRAによる自己ペルソナモデル構築 - Google Colabノートブック
# ==============================================================================
#
# このノートブックは、あなた自身のSlackデータを使って、あなたらしい応答を生成する
# AIモデル（デジタルゴースト）を構築するためのガイドです。
#
# 以下のステップを上から順番に実行していくだけで、LoRAによるファインチューニングの
# 基本を学び、実践することができます。
#
# 作成者: Gemini
#
# ------------------------------------------------------------------------------
# ▼▼▼ 最初のステップ ▼▼▼
# ------------------------------------------------------------------------------
# このノートブックを最大限に活用するために、まずはGPUを有効にしてください。
# メニューバーから [ランタイム] > [ランタイムのタイプを変更] を選択し、
# ハードウェアアクセラレータで「T4 GPU」を選択して保存します。
# ------------------------------------------------------------------------------

# ==============================================================================
# フェーズ 0: 環境構築
# ==============================================================================
#
# 最初に、LoRAでのファインチューニングに必要なライブラリをインストールします。
# これには数分かかることがあります。
#
print("フェーズ0: 必要なライブラリをインストールします...")

!pip install -q transformers datasets peft accelerate bitsandbytes trl sentencepiece

print("ライブラリのインストールが完了しました。")

フェーズ0: 必要なライブラリをインストールします...
ライブラリのインストールが完了しました。


In [5]:
# %%
# ==============================================================================
# フェーズ 1: ベースモデルとトークナイザの準備
# ==============================================================================
#
# ここでは、ファインチューニングの「土台」となる日本語の大規模言語モデル（LLM）と、
# テキストをモデルが理解できる数値に変換する「トークナイザ」を読み込みます。
#
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print("\nフェーズ1: ベースモデルとトークナイザを準備します...")

# ■ モデル名の指定
# ここでは、比較的高性能で扱いやすい日本語モデルを指定します。
# 他のモデル（例: "rinna/japanese-gemma-7b-instruct"）を試すことも可能です。
model_id = "stabilityai/japanese-stablelm-instruct-gamma-7b"

# ■ 量子化の設定
# モデルの重みを4bitで読み込む設定です。これにより、Google Colabの無料GPUでも
# メモリ不足にならずに大規模なモデルを扱うことができます。
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ■ モデルの読み込み
# 指定したモデルIDと量子化設定を使って、モデルをGPUに読み込みます。
# これには数分かかることがあります。
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # 自動的にGPUを割り当てます
    trust_remote_code=True
)

# ■ トークナイザの読み込み
# モデルに対応したトークナイザを読み込みます。
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # パディングトークンを設定

print("ベースモデルとトークナイザの準備が完了しました。")
print("モデル:", model_id)



フェーズ1: ベースモデルとトークナイザを準備します...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ベースモデルとトークナイザの準備が完了しました。
モデル: stabilityai/japanese-stablelm-instruct-gamma-7b


In [ ]:
# %%
# ==============================================================================
# フェーズ 2: 学習データの準備
# ==============================================================================
#
# ここが最も重要なステップです。あなた自身のSlackデータを、モデルが学習できる
# 形式に整形します。
#
# 今回は、以下のような「指示」「入力」「出力」の3つの要素を持つデータ形式を
# 使います。
#
# - 指示 (instruction): モデルに何をしてほしいかを伝える命令文。
# - 入力 (input): 応答を生成するための文脈（Slackのスレッドのやり取りなど）。
# - 出力 (output): モデルに学習させたい、あなた自身の返信。
#
from datasets import Dataset
import pandas as pd
import json

print("\nフェーズ2: 学習データを準備します...")

# ------------------------------------------------------------------------------
# ▼▼▼ あなたの作業エリア ▼▼▼
# ------------------------------------------------------------------------------
#
# ここに、あなた自身のSlackデータを準備します。
# Slackからエクスポートしたデータを、以下の `my_slack_data` のような
# リスト形式に加工してください。
#
# 【加工のヒント】
# 1. Slack APIやエクスポート機能で会話データを取得します。
# 2. PythonのPandasなどを使ってデータを読み込み、ループ処理で
#    「スレッドの文脈（input）」と「あなたの返信（output）」のペアを作ります。
# 3. 最低でも100件以上、できれば300件以上あるとペルソナの再現性が高まります。
#
# ------------------------------------------------------------------------------

# ■ サンプルデータ
# まずは、このサンプルデータで全体の流れを掴んでみましょう。
# 自分のデータが準備できたら、この部分を書き換えてください。
my_slack_data = [
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーA: 「来週の定例会議ですが、アジェンダ案を共有します。何か追加項目はありますか？」",
        "output": "ありがとうございます！拝見しました。可能であれば、先日話していた新機能の進捗についても5分ほど時間をいただけると嬉しいです。"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーB: 「この前の件、調査してみたんですが、原因はサーバー側の設定ミスだったみたいです。すみません…。」",
        "output": "調査ありがとうございます！原因が特定できてよかったです。誰にでもミスはありますし、気にしないでください。すぐに対応してくれて助かりました！"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーC: 「新しいAIの論文読んだ？すごく面白かったよ。」",
        "output": "読みました！特にエージェントが自律的に協調する部分、示唆に富んでますよね。あれを私たちのプロダクトに応用できないか、少し考えてみたくなります。"
    },
    # ... ここにあなたのデータを追加していきます ...
]

# データをDataFrameに変換して確認
df = pd.DataFrame(my_slack_data)
print("--- 学習データのサンプル ---")
display(df)
print("--------------------------")
print(f"データ件数: {len(df)}件")


# ■ プロンプト形式の作成
# モデルが学習しやすいように、データを特定のテキスト形式（プロンプト）に変換します。
def create_prompt(data_point):
    prompt = f"""以下は、タスクを説明する指示と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{data_point["instruction"]}

### 入力:
{data_point["input"]}

### 応答:
{data_point["output"]}"""
    return {"text": prompt}

# ■ データセットの作成
# 用意したデータをHugging FaceのDataset形式に変換し、プロンプトを適用します。
dataset = Dataset.from_pandas(df)
dataset = dataset.map(create_prompt)

print("\n学習データの準備が完了しました。")

In [ ]:

# %%
# ==============================================================================
# フェーズ 3: LoRAによるファインチューニングの実行
# ==============================================================================
#
# いよいよ、準備したデータを使ってモデルのファインチューニングを行います。
# ここでは、PEFT (Parameter-Efficient Fine-Tuning) ライブラリのLoRAを使います。
#
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

print("\nフェーズ3: LoRAによるファインチューニングを開始します...")

# ■ LoRAの設定
# LoRAの挙動を決定する重要なパラメータを設定します。
lora_config = LoraConfig(
    r=16,  # LoRAのランク。大きいほど表現力は増すが、計算量も増える。8, 16, 32あたりが一般的。
    lora_alpha=32, # LoRAのスケーリング係数。rの2倍程度に設定することが多い。
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # 注意機構のどの部分を対象にするか。
    lora_dropout=0.05, # LoRA層のドロップアウト率。
    bias="none",
    task_type="CAUSAL_LM"
)

# ■ モデルの前処理
# 量子化されたモデルをLoRAで学習させるための準備をします。
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# ■ 学習の引数設定
# ファインチューニングの条件を設定します。
training_args = TrainingArguments(
    output_dir="./lora-output",      # 学習結果の出力先ディレクトリ
    num_train_epochs=3,              # 学習のエポック数。データ量に応じて調整。
    per_device_train_batch_size=2,   # バッチサイズ。メモリに応じて調整。
    gradient_accumulation_steps=4,   # 勾配を蓄積するステップ数。バッチサイズを大きくするのと同様の効果。
    optim="paged_adamw_8bit",        # メモリ効率の良いオプティマイザ。
    learning_rate=2e-4,              # 学習率。
    logging_steps=2,                 # ログを出力する間隔。
    fp16=True,                       # 混合精度学習を有効にする。
    # bf16=False, # A100/H100 GPUの場合はTrueにする
    report_to="none",                # レポート先（WandBなど）。今回はオフ。
)

# ■ トレーナーの初期化
# SFTTrainerを使って、学習プロセスを簡単に実行できるようにします。
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    max_seq_length=1024,             # モデルが一度に処理できるトークンの最大長。
    tokenizer=tokenizer,
    args=training_args,
)

# ■ 学習の開始
print("\n--- 学習開始 ---")
trainer.train()
print("--- 学習完了 ---")

# ■ 学習済みモデル（LoRAアダプター）の保存
output_dir = "./lora-persona-model"
trainer.save_model(output_dir)
print(f"学習済みのLoRAアダプターを {output_dir} に保存しました。")


In [ ]:






# %%
# ==============================================================================
# フェーズ 4: 学習済みモデルによる推論（対話テスト）
# ==============================================================================
#
# 学習したペルソナモデルが、実際に「あなたらしい」応答を生成できるか試してみましょう。
#
from peft import PeftModel

print("\nフェーズ4: 学習済みモデルで応答を生成します...")

# ■ 推論の準備
# ベースモデルに学習したLoRAアダプターをマージします。
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = PeftModel.from_pretrained(model, output_dir)
model.eval()

# ■ 応答生成関数
def generate_response(instruction, context):
    # 推論用のプロンプトを作成
    prompt = f"""以下は、タスクを説明する指示と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{instruction}

### 入力:
{context}

### 応答:
"""
    # モデルに入力して応答を生成
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 応答部分だけを抽出
    response = generated_text.split("### 応答:")[1].strip()
    return response

# ------------------------------------------------------------------------------
# ▼▼▼ 対話テストエリア ▼▼▼
# ------------------------------------------------------------------------------
#
# `test_context` の内容を書き換えて、色々な応答を試してみてください。
# あなたのペルソナは再現されていますか？
#
# ------------------------------------------------------------------------------

# ■ テスト実行
test_instruction = "以下の対話の文脈に続いて、あなたらしく返信を生成してください。"
test_context = "ユーザーD: 「急で申し訳ないんだけど、この資料のレビューお願いできないかな？ 明日の朝までだと嬉しいんだけど…。」"

print(f"\n【入力コンテキスト】\n{test_context}")
print("\n【生成された応答】")
response = generate_response(test_instruction, test_context)
print(response)

print("\n--- 対話テスト完了 ---")
print("ノートブックの全プロセスが終了しました。お疲れ様でした！")

フェーズ0: 必要なライブラリをインストールします...
ライブラリのインストールが完了しました。

フェーズ1: ベースモデルとトークナイザを準備します...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 